In [23]:
import numpy as np
import random
import pandas as pd
import copy

from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# World Sense Disambiguation (WSD)

Ce notebook permet d'effectuer une comparaison des performances de deux algorithmes, un d'apprentissage supervisé et un d'apprentissage semi-supervisé, pour la tâche de Word Sense Disambiguation (WSD). 

Les deux méthodes sont les suivants :
- pour la classification supervisée, utilisation d'un MLP
- pour la classification semi-supervisée, utilisation d'un algorithme de constrained K-Means

Plusieurs tests sont effectués en considérant plusieurs mots à désambiguiser pour lesquels les performances de ces deux méthodes sont évaluées. L'objectif final est de comparer ces performances pour déterminer à partir de combien de données annotées la première méthode est préférable à la deuxième et vice versa.



## Extraction des données

La classe Extractor permet d'accéder aux données et de les stocker dans des dictionnaires puis des embeddings. Elle permet de figer les données sur lesquelles les tests de classification seront ensuite faits. Elle comporte les méthodes suivantes :

- `extract_examples()` : pour extraire les données d'entraînement et de test provenant de fichiers XML FSE

- `extract_embeddings()` : pour extraire les embeddings à partir d'un fichier créé en amont, qui ne regroupe que les embeddings nécéssaires à notre jeu de données. Cette méthode est surtout utile pour drastiquement réduire le temps de chargement des embeddings

- `look_up()` : pour effectuer l'étape de look-up avant la classifcation

- `select_examples()` : pour sélectionner des données représentatives lorsqu'on ne considère pas toutes les données annotées. Pour qu'un ensemble de données soit représentatif, il faut que chaque étiquette présente dans les données totales soit présente au moins une fois dans le set d'entraînement sélectionné

- `define_instance()` : pour choisir les données qui seront classifiées, c'est-à-dire quelle proportion des données seront utilisées (80%, 50%, 25%...)


In [24]:
class Extractor :
    
    def __init__(self, data_path, gold_path, embeddings_path, context_size):
        '''
        A partir des données, extrait les examples qui seront utilisés lors de l'apprentissage.

        INPUT
        data_path (str) : chemin pour accéder aux données d'entraînement, au format XML
        gold_path (str) : chemin pour accéder aux numéros de sens correspondant aux données
                          d'entraînement
        embeddings_path (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données
        context_size (int) : taille de la fenêtre du contexte du mot à désambiguiser

        ARGUMENTS DE CLASSE
        self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
                                 utilisation dans une phrase du corpus. 
                                 examples (list) : liste de tous les couples 
                                  ([mots du contexte], numéro de sens) pour un lemme donné
        self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        self.w2emb (dict) : associe chaque mot à son embedding
        self.selected_examples (liste) : liste d'examples qui contient au moins un example de 
                                         chaque sens du lemme choisi
        self.selected_examples_embs (liste) : même liste d'examples, mais avec les contextes 
                                              sous forme d'embedding
        self.X_train (list) : corpus d'apprentissage
        self.X_test (list) : numéros des sens correspondant au corpus d'apprentissage
        self.y_train (list) : corpus de test
        self.y_test (list) : numéros des sens correspondant au corpus d'apprentissage
        self.annotated_examples (list) : liste des données considérées comme annotées 
        self.unannotated_examples (list) : liste des données considérées comme non annotées 
        '''
        
        #récupération des données XML d'apprentissage fournies en argument
        tree = ET.parse(data_path)
        data_file = tree.getroot()

        #récupération des données txt correspondant aux gold class
        gold_file = open(gold_path, "r",encoding="utf-8")
        
        # self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
        #                          utilisation dans une phrase du corpus.
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        #
        # self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        self.w2examples, self.w2senses = self.extract_examples_and_senses(data_file, gold_file, context_size)

        # self.w2emb (dict) : associe chaque mot à l'embedding représentant son contexte sommé
        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding
        self.w2emb = self.extract_embeddings(embeddings_path)

        # liste d'examples qui contient au moins un example de chaque sens du lemme choisi
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples = []
        # même liste d'examples, mais avec les contextes sous forme d'embedding
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples_embs = []
        # liste des examples non selectionnés, dont les contextes sont sous forme d'embedding
        self.unannotated_set = []

        # les corpus seront définis lorsque l'instance du mot à désambiguiser sera choisi
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []

        # liste des données considérées comme annotées 
        self.annotated_examples = []
        # liste des données considérées comme non annotées 
        self.unannotated_examples = []
    
    def extract_examples_and_senses(self, data_file, gold_file, context_size):
        '''
        Extrait les données à partir des fichiers de corpus d'apprentissage et de gold classes.

        INPUT
        data_file (Element): représentation des phrases du corpus d'apprentissage
        gold_file (TextIOWrapper): fichier contenant les numéros de sens (gold class) pour chaque
                                   mot à désambiguiser
        '''
    
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        w2examples = {}
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        w2senses = defaultdict(set)

        # lecture du fichier gold
        gold_file = gold_file.readlines()
        
        # index de parcours dans le fichier gold
        i_gold = 0

        # pour chaque phrase du corpus
        for text_id in data_file:
            for sentence in text_id:

                # recherche de(s) l'indice(s) de(s) l'instance(s) pour savoir où se trouve la fenêtre
                i_instances = []
                for j in range(len(sentence)):
                    if sentence[j].tag == "instance":
                        i_instances.append(j)
                
                # tant qu'il y a des instances à repérer dans la phrase
                while len(i_instances) > 0:

                    # enregistrement du lemme de l'instance
                    instance = sentence[i_instances[0]].attrib["lemma"].lower()
                    
                    # si l'instance n'a pas encore de contexte
                    if instance not in w2examples : 
                        # le crée
                        w2examples[instance] = []

                    context = []
                    # pour chaque mot de la fenêtre
                    for k in range(-context_size, context_size + 1):
                        # si le mot est l'instance, enregistrement de son lemme
                        if k == 0:
                            context.append(instance)
                        # sinon enregistrement du mot du contexte
                        elif len(sentence) > i_instances[0] + k and i_instances[0] + k >= 0:
                            context.append(sentence[i_instances[0] + k].text.lower())
                    
                    # récupération des différents sens possibles dans le fichier gold 
                    gold_class = gold_file[i_gold].split("__")[1].split("_")[1]

                    # le fichiers gold et data ayant les données dans le même ordre, les instances
                    # et les sens peuvent être enregistrés en même temps dans leur dictionnaire
                    # respectif
                    w2senses[instance].add(gold_class)
                    w2examples[instance].append((context, gold_class))
                    
                    # l'instance lue est enlevée de la to-do list et l'index de parcours du fichier 
                    # gold est incrémenté pour passer à l'instance suivante
                    i_instances.pop(0)
                    i_gold += 1

        return w2examples, w2senses
    
    def extract_embeddings(self,path_embeddings) :
        '''
        Récupère les embeddings dans le fichier généré et associe à chaque mot son
        embedding.

        INPUT
        path_embeddings (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données

        OUTPUT
        w2emb (dict) : associe chaque mot à son embedding
        '''

        # lecture du fichier
        file_embeddings = open(path_embeddings , "r", encoding="UTF-8")

        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding du mot
        w2emb = {}

        # pour chaque mot du fichier
        for line in file_embeddings.readlines():

            # séparation du mot et de l'embedding
            splitted_line = line.split(" ")
            word = splitted_line[0]
            embedding = list(map(float,splitted_line[1:]))
            # insertion du mot et de son embedding dans le dictionnaire
            w2emb[word] = embedding

        return w2emb

    def look_up(self, context) :
        '''
        Remplace dans les mots du vecteur de contexte par leur embedding et en fait la
        somme.

        INPUT
        context (list) : liste de taille (size_window*2)+1

        OUTPUT
        context_emb (list) : liste de taille size_embedding (300 ici)
        '''

        # taille d'un embedding : 300
        emb_size = len(list(self.w2emb.values())[0]) 
        # initialisation du contexte sous forme d'embedding
        context_emb = np.zeros(emb_size)

        # pour chaque mot du contexte
        for word in context :
            # s'il est présent dans le dictionnaire d'embeddings
            if word in self.w2emb :
                # il est ajouté à l'embedding représentant le contexte
                context_emb = np.add(context_emb, np.array(self.w2emb[word])) 

        return context_emb
    
    def select_examples(self, examples, senses, sample_size):
        '''
        Choisit des examples représentatifs du corpus selon un nombre imposé de 
        données à choisir.

        INPUT
        examples (list) : liste d'examples pour le mot à désambiguiser
        n_senses (int) : nombre de sens associés à ce mot
        sample_size (float) : quantité des données considérés

        OUTPUT
        selected_examples (list) : liste d'examples qui contient au moins un example 
                                   de chaque sens
        '''

        selected_examples = []
        
        # pour chaque sens, on ajoute un example associé à ce sens, au hasard
        for sense in senses :
            selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),
                                                               examples))))
        
        # ajoute le nombre de données manquantes (non-présentes dans la liste) selectionnées au hasard
        random.shuffle(examples)
        for example in examples:
            if example not in selected_examples and len(selected_examples) < sample_size*len(examples):
                selected_examples.append(example)

        return selected_examples
    
    def select_train_test(self, x, y, senses, data_size, display, train_size = 0.8):
        '''
        Sépare en un train set et test set les examples représentatifs du corpus en mettant 
        au moins une étiquette de chaque sens dans le train set (et le test set si possible)

        INPUT
        x (list) : liste d'examples pour le mot à désambiguiser
        y (list) : liste d'étiquette pour chaque example de x
        senses (list) : numéros des sens associés à ce mot
        train_size (float) : quantité des données d'entraînement considérés
        '''

        # SELECTION DES DONNEES REPRESENTATIVES DU TRAIN

        # sélection au hasard d'un contexte et de son étiquette pour chaque sens
        selected_train = [random.choice(list(filter((lambda example: example[1] == sense),
                                                    zip(x,y)))) for sense in senses]
            
        # SELECTION DES DONNEES REPRESENTATIVES DU TEST

        # fait de même pour le corpus de test, lorsque possible, en ne considérant pas
        # ceux déjà sélectionnés pour le train
        selected_test = []
        for sense in senses:
            added = False
            for example in zip(x,y):
                for train_ex in selected_train:
                    # vérifie que l'example n'a pas déjà été sélectionné dans le train et
                    # le test pour ne pas faire de doublon
                    if example[1] == sense and example[1] == train_ex[1] and not(np.array_equal(example[0], train_ex[0])):
                        selected_test.append(example)
                        added = True
                        break
                if added == True:
                    break

        # SELECTION DES DONNEES RESTANTES DU TRAIN ET DU TEST

        # effectue un split sur le reste d'examples disponibles
        remaining_X = []
        remaining_y = []
        selected_train_copy = copy.deepcopy(selected_train)
        selected_train_copy.extend(selected_test)
        for contexte, sens in zip(x,y):
            to_add = True
            for ex in selected_train_copy:
                # vérifie que l'example n'a pas déjà été sélectionné dans le train et le
                # test pour ne pas faire de doublon
                if np.array_equal(contexte, ex[0]) and sens == ex[1]:
                    to_add = False
                    break
            if to_add:
                remaining_X.append(contexte)
                remaining_y.append(sens)

        try:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(remaining_X, remaining_y,train_size = train_size)
    
            # ajout des données représentatives dans les sets correspondant
            selected_X_train, selected_y_train = zip(*selected_train)
            selected_X_test, selected_y_test = zip(*selected_test)
            
            self.X_train.extend(selected_X_train)
            self.X_test.extend(selected_X_test)
            self.y_train.extend(selected_y_train)
            self.y_test.extend(selected_y_test)
            
        except:
            if display:
                print(f"/!\ ERREUR /!\ La taille des données est trop faible. L'opération train_test_split() est impossible.")                                                                     
            self.X_train, self.X_test, self.y_train, self.y_test = set(), set(), set(), set()
                     
    def define_instance(self, instance, data_size, display):
        '''
        Permet de définir l'instance du mot à désambiguiser et les examples à utiliser. 
        Fixe les derniers arguments de classe selon l'instance choisie.
        Contraint le choix de classification selon le nombre de données considéré.

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float): quantité de données à considérer
        display (bool, optional): affiche une trace des options choisies
        '''

        # SET DE DONNEES ANNOTEES

        # choisit des examples d'entraînement représentatifs du corpus selon le nombre
        # imposé de données
        self.annotated_examples = self.select_examples(self.w2examples[instance],
                                                       self.w2senses[instance],
                                                       data_size)
        # liste de tous les contextes sous forme d'embedding
        X_annotated = [self.look_up(context) for context, gold in self.annotated_examples]
        # liste de tous les numéros de sens
        y_annotated = [gold for context, gold in self.annotated_examples]
        # le set des données annotées est une liste des examples représentatifs sous forme d'embeddings
        self.selected_examples_embs = list(zip(X_annotated,y_annotated))

        # SET DE DONNEES NON ANNOTEES

        # garde en mémoire les examples considérés comme non-annotés
        self.unannotated_examples = [example for example in self.w2examples[instance] 
                                     if example not in self.annotated_examples]        
        # liste de tous les contextes sous forme d'embedding
        X_unannotated = [self.look_up(context) for context,gold in self.unannotated_examples]
        # liste de tous les numéros de sens
        y_unannotated = [gold for context,gold in self.unannotated_examples]
        # le set des données non-annotées est une liste des examples sous forme d'embeddings
        self.unannotated_set = list(zip(X_unannotated,y_unannotated))

        # SET DE DONNEES A CLASSIFIER PAR MLPCLASSIFIER_WSD
        
        # création des corpus d'entrainement et de test
        self.select_train_test(X_annotated, y_annotated, self.w2senses[instance],display,data_size)
            
        # affichage des informations relatives à l'instance choisie pour MLPClassifier_WSD
        if display :
            self.classifier_display_at_instance_time(instance, data_size)
            
    
    def classifier_display_at_instance_time(self, instance, data_size):
        '''
        Affiche les informations relatives à l'instance choisie pour MLPClassifier_WSD

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float) : quantité de données à considérer
        '''
        
        print("--> Instance :", instance)
        print(f'{data_size*100}% des données annotées considérées')
        print("Nombre de données d'entraînement : ", len(self.X_train))
        print("Etiquettes possibles pour cette instance : ", self.w2senses[instance])
        if self.y_train:
            print("Etiquettes présentes dans les données d'entraînement :", Counter(self.y_train))
            

    def kmeans_display_at_instance_time(self, instance, size):
        ''' 
        Affiche des informations relatives à l'instance choisie pour K_Means

        INPUT
        instance (str) : mot à désambiguiser
        num_correct (int) : nombre de prédictions correctes
        num_examples (int) : nombre d'examples total
        size (float) : quantité de données à considérer 
        '''
        print("--> Instance :",instance)
        print(f"Proportion de données annotées considérées sur le corpus: {size*100} %")
        y, most_frequent_sense, occurrence_of_most_frequent_sense = self.get_most_frequent_sense()
        print(f"Répartition des sens: {Counter(y)}")
        print(f"Le sens le plus fréquent pour '{instance}' est le sens {most_frequent_sense} avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
        #print(f"Pour '{instance}', l'accuracy est de {round(num_correct/num_examples,2)} %")
        print()

    def get_most_frequent_sense(self):
        y = [gold for tensor,gold in self.selected_examples_embs] + [gold for tensor,gold in self.unannotated_set] 
        most_frequent_sense = max(y,key=y.count)
        occurrence_of_most_frequent_sense = y.count(most_frequent_sense)/len(y)
        return y, most_frequent_sense, occurrence_of_most_frequent_sense

    def get_sets(self):
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def get_annotated_and_unannotated_sets(self):
        return self.selected_examples_embs, self.unannotated_set
        
    def get_annotated_examples(self):
        return self.annotated_examples
    
    def get_unannotated_examples(self):
        return self.unannotated_examples
    
    def get_embs(self):
        return self.selected_examples_embs

In [25]:
# chemin du corpus annoté
data_path = "../donnees/FSE-1.1-191210/FSE-1.1.data.xml"
# chemin pour récupérer les gold class du corpus annoté
gold_path = "../donnees/FSE-1.1-191210/FSE-1.1.gold.key.txt"
# choix de la fenêtre du contexte
context_size = 4
# chemin pour récupérer les embeddings afin de faire l'opération look-up
# Les embeddings sont extraits de fasttext
embeddings_path = "../donnees/embeddings.txt"
# choix de l'instance à tester pour la classification
instance = "aboutir"

In [26]:
# exemple d'utilisation d'Extractor avec le verbe "aboutir" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance(instance, 0.8, display=True)
print("\nexemple d'example avec contexte et gold class :",ext.get_annotated_examples()[0])
print("\nau format embedding : ",ext.get_embs()[0])

--> Instance : aboutir
80.0% des données annotées considérées
Nombre de données d'entraînement :  31
Etiquettes possibles pour cette instance :  {'2', '4', '1', '3'}
Etiquettes présentes dans les données d'entraînement : Counter({'3': 25, '4': 4, '2': 1, '1': 1})

exemple d'example avec contexte et gold class : (['les', 'produits', 'ainsi', 'formés', 'aboutir', 'dans', 'des', 'vésicules', 'destinées'], '2')

au format embedding :  (array([ 8.6700e-02, -8.6500e-02,  1.1530e-01, -1.9800e-02, -5.4330e-01,
       -1.1660e-01,  4.0650e-01, -3.2600e-01,  1.4230e-01, -9.7700e-02,
        1.2140e-01, -3.3560e-01,  1.1430e-01,  1.2610e-01,  4.3000e-02,
        6.0600e-02,  8.3600e-02, -4.5000e-02,  2.9500e-02,  9.6500e-02,
       -3.4000e-03, -1.3260e-01, -7.7800e-02, -2.4500e-02, -7.0000e-04,
        1.6780e-01, -8.2870e-01, -1.3090e-01,  5.3910e-01, -1.6000e-03,
       -1.1190e-01,  9.0000e-04, -3.9980e-01,  3.1190e-01,  1.2870e-01,
        1.8070e-01,  5.4570e-01, -4.8000e-03, -1.1460e-01, -


## Classification supervisée

L'utilisation d'un Classifieur MLP permet de prédire le sens des verbes de façon supervisée. Il s'agit de la première méthode proposée pour la tâche de WSD.

Les données utilisées sont les données annotées extraites du fichier FSE grâce à l'Extractor pour un lemme donné. Un MLPClassifier_WSD doit donc être créé pour chaque verbe dont on veut la prédiction.

MLPClassifier_WSD comporte les méthodes suivantes :
- `classify()` : pour prédire le sens du lemme associé au classifieur et calculer l'accuracy de cette prédiction


In [27]:
class MLPClassifier_WSD :
    
    def __init__(self, X_train, X_test, y_train, y_test) :
        '''
        A partir des données, prédit le sens d'un lemme et évalue la prédiction

        ARGUMENTS DE CLASSE
        self.X_train (list) : liste des contextes d'entraînement
        self.y_train (list) : liste des gold class d'entraînement
        self.X_test (list) : liste des contextes de test
        self.y_test (list) : liste des gold class de test
        self.accuracy (float)
        self.macro_average (float)
        self.baseline_accuracy (float)
        self.baseline_macro_average (float)
        
        '''
        # ensembles d'entraînement et de test provenant de l'Extractor pour un lemme donné
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 

        # prédiction du modèle
        self.y_pred = []
        # accuracy de la prédiction
        self.accuracy = 0
        #macro_average de la prédiction
        self.macro_average = 0
        #baselines : most frequent sense
        #si la quantité de donées sélectionnées est suffisante
        if y_train :
            count_gold = Counter(y_train)
            #la baseline de l'accuracy : on divise le nombre d'étiquette 
            # dont le gold correspond au sens le plus fréquent par le nombre d'étiquettes prédites
            self.baseline_accuracy = round(max(count_gold.items(),key = lambda item:item[1])[1]/len(y_train),2)
            #la baseline de la macro-average : on divise 1 (qui correspond à l'accuracy du sens le plus fréquent) 
            # par le nombre d'étiquettes possibles (les autres ont donc une accuracy de 0)
            self.baseline_macro_average = 1/len(count_gold)
        else :
            self.baseline_accuracy = -1.0
            self.baseline_macro_average = -1.0
            
    def classify(self, display=False) :
        ''' 
        Permet de prédire le sens du verbe.

        INPUT
        display (bool, optional) : affiche une trace des options choisies
        '''
        try:
            clf = MLPClassifier(random_state=1, hidden_layer_sizes=(100,)) 
            # entraînement
            clf.fit(self.X_train, self.y_train)
            # prédiction
            self.y_pred = clf.predict(self.X_test)

            # calcul de l'accuracy
            self.accuracy = accuracy_score(self.y_pred, self.y_test)
            
            # calcul du macro-average
            sense2accuracy = {}
            # on récupère les sens possibles 
            senses = Counter(self.y_test).keys()
            #pour chaque sens possible
            for sense in senses:
                #on récupère la liste des golds et des predictions (bonnes ou fausses) pour ce sens
                golds_and_preds = [(gold,pred) for gold,pred in zip(self.y_test,self.y_pred) if gold == sense]
                #on divise le nombre de sens correctement prédits par le nombre de golds pour ce sens
                sense2accuracy[sense] = sum([1 for gold,pred in golds_and_preds if gold == pred])/len(golds_and_preds)

            self.macro_average = sum(sense2accuracy.values())/len(sense2accuracy) 
            
            # si une trace est souhaitée
            if display :
                print("prediction :", self.y_pred)
                print("gold :", self.y_test)
                print("accuracy score : ", round(self.accuracy,2))
                print("baseline accuracy score :",self.baseline_accuracy)
                print("macro average precision : ", round(self.macro_average,2))
                print("baseline macro average precision :",round(self.baseline_macro_average,2),"\n")
        except:
             print("/!\ ERREUR /!\: La quantité de données fournies est insuffisante: formation du test set impossible")
        

In [28]:
# exemple d'utilisation de MLPClassifieur_WSD avec le verbe "demeurer" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("demeurer", 0.8, display=True)
X_train, X_test, y_train, y_test = ext.get_sets()

Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
Clf.classify(display=True)

--> Instance : demeurer
80.0% des données annotées considérées
Nombre de données d'entraînement :  28
Etiquettes possibles pour cette instance :  {'6', '2', '3', '4'}
Etiquettes présentes dans les données d'entraînement : Counter({'4': 18, '3': 5, '2': 4, '6': 1})
prediction : ['4' '4' '4' '4' '4' '4' '2' '3' '4']
gold : ['4', '4', '4', '4', '4', '4', '2', '3', '4']
accuracy score :  1.0
baseline accuracy score : 0.64
macro average precision :  1.0
baseline macro average precision : 0.16 



c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pour pouvoir comparer les performances de ce Classifieur MLP avec celles de l'apprentissage semi-supervisé, nous devons créer un classifieur par verbe à désambiguiser et choisir un pas de descente pour la quantité de données annotées considérées.

Pour chaque classifieur et chaque quantitée de données considérées, on effectue *n_repeat* classifications pour obtenir une accuracy moyenne représentative du classifieur. 

Par conséquent, pour :
- n_repeat = 5
- step = 0.25

nous obtiendrons pour chaque classifieur une liste d'accuracies correspondant à :
- la moyenne des accuracies de 5 prédictions 
- pour 100%, 75%, 50% et 25% des données.

In [29]:
def get_mean_precisions(instance, data_size, n_repeat, display=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    display (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean_acc (float): moyenne des accuracies calculées
    mean_mac (float) : moyenne des macro_average calculés
    baseline_accuracy (float) : baseline de l'accuracy
    baseline_macro_average (float) : baseline du macro_average
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
    
    #liste de macro_average à chaque répétition
    macro_averages = []
        
    for n in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        X_train, X_test, y_train, y_test = ext.get_sets()
        
        # si les sets ne sont pas vides i.e que la quantité de données sélectionnées est suffisante
        if y_train :
            Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
            Clf.classify()
                
            accuracies.append(Clf.accuracy)
            mean_acc = sum(accuracies)/len(accuracies)
            
            macro_averages.append(Clf.macro_average)
            mean_mac = sum(macro_averages)/len(macro_averages)
            
            baseline_accuracy = Clf.baseline_accuracy
            
            baseline_micro_average = Clf.baseline_macro_average
        
        # lorsque la quantité de données sélectionnées est trop faible
        else :
            accuracies.append(-1.0)
            mean_acc = -1.0
            macro_averages.append(-1.0)
            mean_mac = -1.0
            baseline_accuracy = -1.0
            baseline_micro_average = -1.0
            
    # si la trace est souhaitée
    if display :
        print(f'{data_size*100}% des données annotées considérées')
        print("instance :", instance)
        print("accuracies :", accuracies)
        print("accuracy moyenne :", mean_acc)
        print("baseline acuracy score:",baseline_accuracy)
        print("macro average precisons :",macro_averages)
        print("macro average precision moyenne :", mean_mac)
        print("baseline macro average precision :", baseline_micro_average,"\n")

    return mean_acc, mean_mac, baseline_accuracy, baseline_micro_average

def get_precisions(instances, step, n_repeat, display=True):
    '''
    - Permet d'obtenir des precisions moyennes par lemme pour une certaine quantité 
      de données considérée.
    - Produit des fichiers csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes
    instance2acc (dict) : associe à chaque instance sa liste de macro_average moyens
    instance2acc (dict) : associe à chaque instance sa liste de baselines d'accuracies
    instance2mac (dict) : associe à chaque instance sa liste de baseline de macro-average
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    instance2mac = {instance : [] for instance in instances}
    instance2acc_base = {instance : [] for instance in instances}
    instance2mac_base = {instance : [] for instance in instances}
    
    moyennes_acc = []
    moyennes_mac = []
    moyennes_acc_base = []
    moyennes_mac_base = []
    
    data_sizes =[]
            
    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 1.0 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de ses precisions
            mean_acc, mean_mac, accuracy_baseline, macro_average_baseline = get_mean_precisions(instance, data_size, n_repeat, display)
            
            instance2acc[instance].append(mean_acc)
            instance2mac[instance].append(mean_mac)
            
            #ajout de la baseline pour l'accuracy
            instance2acc_base[instance].append(accuracy_baseline)
        
            #ajout de la basleine pour la micro-average
            instance2mac_base[instance].append(macro_average_baseline)
        
        # on boucle sur les valeurs obtenues pour obtenir leur moyenne
        accuracies = [accuracies[i] for accuracies in instance2acc.values() if accuracies[i] != -1.0]
        moyennes_acc.append(sum(accuracies)/len(accuracies))
        
        macro_averages = [macro[i] for macro in instance2mac.values() if macro[i] != -1.0]
        moyennes_mac.append(sum(macro_averages)/len(macro_averages))
        
        accuracies_base = [acc_base[i] for acc_base in instance2acc_base.values() if acc_base[i] != -1.0]
        moyennes_acc_base.append(sum(accuracies_base)/len(accuracies_base))
        
        macro_averages_base = [mac_base[i] for mac_base in instance2acc_base.values() if mac_base[i] != -1.0]
        moyennes_mac_base.append(sum(macro_averages_base)/len(macro_averages_base))
  
    # ajout d'une clé moyennes pour avoir les moyennes de toutes les precisions moyennes pour chaque instance
    # pour chaque pas
    instance2acc["moyennes"] = moyennes_acc
    instance2mac["moyennes"] = moyennes_mac
    instance2acc_base["moyennes"] = moyennes_acc_base
    instance2mac_base["moyennes"] = moyennes_mac_base
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes
    instance2mac["data_sizes"] = data_sizes
    instance2acc_base["data_sizes"] = data_sizes 
    instance2mac_base["data_sizes"] = data_sizes

    # export en csv des résultats
    df1 = pd.DataFrame(instance2acc)
    df1.set_index("data_sizes")
    df1.to_csv(f"results/classifieur_accuracies_par_lemme_{n_repeat}_repet.csv")
    
    df2 = pd.DataFrame(instance2mac)
    df2.set_index("data_sizes")
    df2.to_csv(f"results/classifieur_macro_average_par_lemme_{n_repeat}_repet.csv")
    
    df3 = pd.DataFrame(instance2acc_base)
    df3.set_index("data_sizes")
    df3.to_csv(f"results/classifieur_accuracy_baseline_par_lemme_{n_repeat}_repet.csv")
    
    df4 = pd.DataFrame(instance2mac_base)
    df4.set_index("data_sizes")
    df4.to_csv(f"results/classifieur_macro_average_baseline_par_lemme_{n_repeat}_repet.csv")
    
    return instance2acc, instance2mac, instance2acc_base, instance2mac_base  

In [30]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une precision moyenne
n_repeat = 5

In [31]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())
# calcul des precisions
get_precisions(instances, step, n_repeat)

100.0% des données annotées considérées
instance : aboutir
accuracies : [0.9090909090909091, 0.8181818181818182, 0.8181818181818182, 0.8181818181818182, 0.9090909090909091]
accuracy moyenne : 0.8545454545454545
baseline acuracy score: 0.79
macro average precisons : [0.75, 0.6944444444444444, 0.5, 0.5, 0.5]
macro average precision moyenne : 0.5888888888888889
baseline macro average precision : 0.1975 

100.0% des données annotées considérées
instance : investir
accuracies : [0.4166666666666667, 0.75, 0.75, 0.5833333333333334, 0.5833333333333334]
accuracy moyenne : 0.6166666666666667
baseline acuracy score: 0.71
macro average precisons : [0.20833333333333334, 0.5, 0.4444444444444444, 0.3333333333333333, 0.3333333333333333]
macro average precision moyenne : 0.3638888888888888
baseline macro average precision : 0.1775 

100.0% des données annotées considérées
instance : traduire
accuracies : [0.9166666666666666, 0.5, 0.9166666666666666, 0.5833333333333334, 0.9166666666666666]
accuracy moye

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

100.0% des données annotées considérées
instance : justifier
accuracies : [0.8181818181818182, 0.8181818181818182, 0.45454545454545453, 0.8181818181818182, 0.7272727272727273]
accuracy moyenne : 0.7272727272727273
baseline acuracy score: 0.9
macro average precisons : [0.45, 0.45, 0.2777777777777778, 0.45, 0.4444444444444444]
macro average precision moyenne : 0.4144444444444444
baseline macro average precision : 0.225 

100.0% des données annotées considérées
instance : viser
accuracies : [1.0, 1.0, 1.0, 1.0, 1.0]
accuracy moyenne : 1.0
baseline acuracy score: 0.5
macro average precisons : [1.0, 1.0, 1.0, 1.0, 1.0]
macro average precision moyenne : 1.0
baseline macro average precision : 0.25 



c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


100.0% des données annotées considérées
instance : prononcer
accuracies : [0.38461538461538464, 0.5384615384615384, 0.46153846153846156, 0.5384615384615384, 0.38461538461538464]
accuracy moyenne : 0.4615384615384615
baseline acuracy score: 0.35
macro average precisons : [0.3333333333333333, 0.4833333333333333, 0.3833333333333333, 0.45, 0.3]
macro average precision moyenne : 0.39
baseline macro average precision : 0.06999999999999999 

100.0% des données annotées considérées
instance : accomplir
accuracies : [0.6923076923076923, 0.7692307692307693, 0.7692307692307693, 0.7692307692307693, 0.6153846153846154]
accuracy moyenne : 0.7230769230769231
baseline acuracy score: 0.86
macro average precisons : [0.225, 0.25, 0.25, 0.25, 0.25]
macro average precision moyenne : 0.24500000000000002
baseline macro average precision : 0.215 



...

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


50.0% des données annotées considérées
instance : arriver
accuracies : [0.625, 0.375, 0.375, 0.5, 0.5]
accuracy moyenne : 0.475
baseline acuracy score: 0.55
macro average precisons : [0.375, 0.25, 0.29166666666666663, 0.25, 0.2]
macro average precision moyenne : 0.2733333333333333
baseline macro average precision : 0.09166666666666667 

50.0% des données annotées considérées
instance : chercher
accuracies : [0.2857142857142857, 0.2857142857142857, 0.42857142857142855, 0.42857142857142855, 0.5714285714285714]
accuracy moyenne : 0.4
baseline acuracy score: 0.44
macro average precisons : [0.3333333333333333, 0.27777777777777773, 0.19999999999999998, 0.4444444444444444, 0.6666666666666666]
macro average precision moyenne : 0.3844444444444444
baseline macro average precision : 0.11 

50.0% des données annotées considérées
instance : composer
accuracies : [1.0, 1.0, 1.0, 1.0, 1.0]
accuracy moyenne : 1.0
baseline acuracy score: 0.8
macro average precisons : [1.0, 1.0, 1.0, 1.0, 1.0]
macro ave

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


25.0% des données annotées considérées
instance : aider
accuracies : [0.4, 0.75, 0.2, 0.2, 0.25]
accuracy moyenne : 0.36
baseline acuracy score: 0.56
macro average precisons : [0.5, 0.6666666666666666, 0.25, 0.25, 0.3333333333333333]
macro average precision moyenne : 0.39999999999999997
baseline macro average precision : 0.11200000000000002 

25.0% des données annotées considérées
instance : poursuivre
accuracies : [0.2, 0.25, 0.25, 0.25, 0.2]
accuracy moyenne : 0.22999999999999998
baseline acuracy score: 0.5
macro average precisons : [0.25, 0.3333333333333333, 0.3333333333333333, 0.16666666666666666, 0.25]
macro average precision moyenne : 0.26666666666666666
baseline macro average precision : 0.1 

25.0% des données annotées considérées
instance : profiter
accuracies : [1.0, 1.0, 0.5, 1.0, 1.0]
accuracy moyenne : 0.9
baseline acuracy score: 0.8
macro average precisons : [1.0, 1.0, 0.3333333333333333, 1.0, 1.0]
macro average precision moyenne : 0.8666666666666668
baseline macro averag

({'aboutir': [0.8545454545454545, 0.8666666666666668, 0.8, 0.85],
  'investir': [0.6166666666666667,
   0.6199999999999999,
   0.5904761904761904,
   0.56],
  'traduire': [0.7666666666666666, 0.76, 0.5714285714285714, 0.76],
  'témoigner': [0.8333333333333334, 0.8, 0.8285714285714285, 0.7],
  'juger': [0.5999999999999999, 0.65, 0.6599999999999999, 0.7999999999999999],
  'justifier': [0.7272727272727273,
   0.7777777777777777,
   0.7333333333333333,
   0.6666666666666666],
  'viser': [1.0, 1.0, 0.8571428571428571, 0.85],
  'prononcer': [0.4615384615384615,
   0.41818181818181815,
   0.35,
   0.4333333333333333],
  'accomplir': [0.7230769230769231,
   0.6599999999999999,
   0.6690476190476191,
   0.62],
  'convenir': [0.5692307692307692,
   0.5454545454545454,
   0.55,
   0.44000000000000006],
  'acquérir': [0.6714285714285714,
   0.510909090909091,
   0.5142857142857142,
   0.51],
  'achever': [0.8545454545454545, 0.7555555555555555, 0.6666666666666667, 0.6],
  'observer': [0.6153846153

## Classification semi-supervisée

In [32]:
class Constrained_K_Means():
    ''' 
    Classifieur K-means pour un mot particulier
    '''

    def __init__(self, annotated_examples, not_annotated_examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        INPUT
        annotated_examples (list) : liste des exemples qui seront considérés
                                     comme annotés pour la construction des premiers
                                     centroïdes
        non_annotated_examples (list) : liste des exemples à classifier 

        ARGUMENTS DE CLASSE
        self.annotated_examples
        self.not_annotated_examples
        self.k (int) : nombre de clusters pour ce lemme
        self.centroids (list) : liste des centroïdes sous forme d'embeddings
        self.cluster2sense (list) : associe chaque cluster (indice de la liste)
                                    à un sens
        '''

        self.annotated_examples = annotated_examples

        # les gold class ne seront utilisées que pour calculer l'accuracy
        self.not_annotated_examples = not_annotated_examples

        # initialisation de clusters : tous les examples non annotés sont associés 
        # au cluster 0
        self.clusters = np.zeros(len(not_annotated_examples))

        # détermine le nombre de sens possibles k (donc le nombre de clusters) 
        # à l'aide des données annotées, qui représentent tous les sens possibles
        self.k = len(set([example[1] for example in self.annotated_examples]))

        # initialisation des centroïdes : pour chaque sens, le centroïde 
        # correspond à la moyenne des embeddings des exemples annotés
        # Ainsi, chaque centroïde représente un sens
        self.centroids, self.cluster2sense = self.make_centroids(self.annotated_examples)


    def make_centroids(self, data):
        '''
        Calcule les coordonnées des centroïdes

        OUTPUT
        centroids (list) : liste des coordonnées des centroïdes dans l'ordre
                           des clusters
        cluster2sense (list) : à chaque indice correpond au numéro d'un sens
        '''

        centroids = []
        # clé : numéro du cluster (ie indice de la liste)
        # valeur : numéro du sens associé
        cluster2sense = []
        # regroupe les sens présents dans le corpus
        senses = set([example[1] for example in data])
        # pour chaque sens
        for sense in senses:
            # on récupère les examples correspondant à ce sens
            examples_sense = [example[0] for example in data if example[1] == sense]
            # le centroide correspond à la moyenne de tous ces examples
            centroids.append(np.mean(examples_sense, axis=0))
            # ajoute le numéro du sens
            cluster2sense.append(sense)

        return centroids, cluster2sense


    def learn_clusters(self):
        '''
        Algorithme d'apprentissage de K-Means : met à jour les coordonnées
        des centroïdes après chaque annotation d'exemple
        '''

        # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE
         
        # pour chaque couple (indice, coordonnées) dans les examples
        for i, example in enumerate(self.not_annotated_examples):
            # initialisation de la distance minimum à l'infini
            min_dist = float('inf')
            # pour chaque couple (indice, coordonnées) dans les centroides
            for j, centroid in enumerate(self.centroids):
                # calcul de la distance entre cet example et ce centroide
                d = np.sqrt(np.sum(np.square(centroid - example[0])))
                # si une distance plus faible est trouvée
                if min_dist > d:
                    # la distance ainsi que le centroide sont stockés
                    min_dist = d
                    self.clusters[i] = j
                
            # CALCUL ET MISE A JOUR DES NOUVEAUX CENTROIDES
            
            # donne l'étiquette prédite à cet exemple
            predicted_example = (self.not_annotated_examples[i][0], self.cluster2sense[int(self.clusters[i])])
            # fusionne les données annotées avec cette donnée prédite
            data = self.annotated_examples
            data.append(predicted_example)
            # pour mettre à jour les centroides
            self.make_centroids(data)

    def accuracy(self): 
        '''
        OUTPUT
        - correct (int) : nombre de fois où le sens prédit est le bon
        - len(self.not_annotated_examples) (int): nombre total d'examples non annotés
        - predicted_sense2count (dict): dictionnaire qui associe à chaque sens le nombre de fois où il a été prédit
        '''

        
        sense2f1 = {}
        correctly_predicted_sense2count = {}
        
        for sense in set([example[1] for example in self.annotated_examples]):
            true_positive = 0
            false_negative = 0
            false_positive = 0
            for i in range(len(self.not_annotated_examples)):
                if sense in self.cluster2sense:
                    expected_sense = self.not_annotated_examples[i][1]
                    predicted_sense = self.cluster2sense[int(self.clusters[i])]
                    # si le sens attendu est le même que le sens que l'on recherche, 
                    # alors on va savoir si le sens prédit est correct ou non
                    if expected_sense == sense:
                        # si le sens prédit est le bon, alors on a un vrai positif
                        if predicted_sense == sense:
                            true_positive += 1
                            correctly_predicted_sense2count[predicted_sense] = correctly_predicted_sense2count.get(predicted_sense, 0) + 1
                        # sinon, on a un faux négatif
                        else:
                            false_negative += 1
                    # si le sens attendu n'est pas le même que le sens que l'on recherche,
                    # alors on cherche à savoir si le sens prédit est le même que le sens recherché
                    # (dans ce cas c'est un faux positif)
                    else:
                        if predicted_sense == sense:
                            false_positive += 1
                else: 
                    sense2f1[sense] = 0 

            precision, recall = 0, 0
            if true_positive + false_positive != 0:
                precision = true_positive / (true_positive + false_positive)
            """else: 
                precision = 0"""
            if true_positive + false_negative != 0:
                recall = true_positive / (true_positive + false_negative)
            '''else:
                recall = 0'''
            if precision and recall :
                sense2f1[sense] = 2 * (precision * recall) / (precision + recall)
            else:
                sense2f1[sense] = 0
            

        
        return sense2f1, len(self.not_annotated_examples), correctly_predicted_sense2count

    """correct = 0
        correctly_predicted_sense2count = {}
        for i in range(len(self.not_annotated_examples)):
            expected_sense = self.not_annotated_examples[i][1]
            predicted_sense = self.cluster2sense[int(self.clusters[i])]
            # on compte le nombre de fois où le sens prédit est le bon (sera utilisé pour calculer la micro-average)
            if expected_sense == predicted_sense:
                correct += 1
                # on compte le nombre de fois où chaque sens est correctement prédit (sera utilisé pour calculer la macro-average)
                correctly_predicted_sense2count[predicted_sense] = correctly_predicted_sense2count.get(predicted_sense, 0) + 1
    return correct, len(self.not_annotated_examples), correctly_predicted_sense2count"""

In [33]:
def process_annotated_and_unannotated_sets(instance, data_size):
    '''
    - Dernière étape de la préparation des données pour le clustering.
    INPUT
    instance (str): mot à désambiguiser
    data_size (float): quantité de données à considérer

    OUTPUT
    annotated_set (list): liste des données annotées contenant des vecteurs numpy
    unannotated_set (list): liste des données non annotées contenant des vecteurs numpy
    
    '''

    ext.define_instance(instance, data_size, False)
        
    annotated_set, unannotated_set = ext.get_annotated_and_unannotated_sets()
    annotated_set = [(np.array(embedding),sense) for embedding,sense in annotated_set]
    unannotated_set = [(np.array(embedding),gold) for embedding,gold in unannotated_set]
    
    return annotated_set, unannotated_set


def make_csv_per_instance(instance, n_repeat, data_size, macro_average, micro_average, averages2score, n_repeat_most_frequent_senses): 
    # on ouvre le fichier csv pour ajouter une ligne avec les moyennes obtenues
    df = pd.read_csv(f"results/kmeans_accuracies_{instance}_for_{n_repeat}_repetitions.csv")
    df.loc[len(df)] = [data_size, macro_average, micro_average, n_repeat_most_frequent_senses]
    df.to_csv(f"results/kmeans_accuracies_{instance}_for_{n_repeat}_repetitions.csv", index=False)

    # on stocke les moyennes obtenues pour chaque instance
    averages2score["macro_average"].append(macro_average)
    averages2score["micro_average"].append(micro_average)
    averages2score["most_frequent_sense"].append(n_repeat_most_frequent_senses)

    return averages2score

def get_mean_metrics(n_repeat_macro_averages, n_repeat_micro_averages, display): 
    mean_macro_average_score = sum(n_repeat_macro_averages)/len(n_repeat_macro_averages)
    mean_micro_averages = sum(n_repeat_micro_averages)/len(n_repeat_micro_averages)
    if display:
        print(f"Macro-average : {round(mean_macro_average_score,2)}")
        print(f"Micro-average : {round(mean_micro_averages,2)}")
        print("\n--------------------------------------------------\n")
    return round(mean_macro_average_score,2), round(mean_micro_averages,2)

def create_kmeans_mean_accuracies_csv(mean_averages2score, mean_macro_averages_for_kmeans, mean_micro_averages_for_kmeans, data_sizes, n_repeat):
    # ajout d'une clé macro_average_moyenne_du_kmeans pour avoir les moyennes de toutes les accuracy moyennes pour chaque instance
    # pour chaque pas
    mean_averages2score["macro_average_moyenne_du_kmeans"] = mean_macro_averages_for_kmeans
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    mean_averages2score["micro_average_moyenne_du_kmeans"] = mean_micro_averages_for_kmeans 
    mean_averages2score["data_sizes"] = data_sizes
    # export en csv des résultats
    df = pd.DataFrame(mean_averages2score)
    df.set_index(["data_sizes"], inplace=True)
    df.to_csv(f"results/kmeans_mean_accuracies_for_{n_repeat}_repetitions.csv")

In [34]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5
# nombre d'itérations maximal
max_iter = 10

In [35]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5
# nombre d'itérations maximal
max_iter = 10

instances = list(ext.w2examples.keys())[:3]

averages2score = {"macro_average":[], "micro_average":[], "most_frequent_sense":[]}
mean_averages2score = {"macro_average_moyenne_du_kmeans": 0, "micro_average_moyenne_du_kmeans":0}

mean_macro_averages_for_kmeans = []
mean_micro_averages_for_kmeans = []
data_sizes =[]
n_repeat_most_frequent_senses = []
    
# pour chaque quantité de données observée
for i in range(round(1.0/step)): 
    # calcul de la taille des données avec le pas de descente (75%, 50%, 25%, 1%)
    data_size = 0.75 - (step*float(i))
    data_sizes.append(data_size)

    for instance in instances: 
        
        if i == 0:
            # on crée un dataframe pour chaque instance
            df = pd.DataFrame(columns=["data_sizes", "macro_average", "micro_average", "most_frequent_sense"])
            df.to_csv(f"results/kmeans_accuracies_{instance}_for_{n_repeat}_repetitions.csv", index=False)
            
        n_repeat_micro_averages = []
        n_repeat_macro_averages = []
        for _ in range(n_repeat):
            annotated_set, unannotated_set = process_annotated_and_unannotated_sets(instance, data_size)
            k_Means = Constrained_K_Means(annotated_set, unannotated_set)
            k_Means.learn_clusters()
            #correct, num_examples, correctly_predicted_sense2count = k_Means.accuracy()
            sense2f1, num_examples, correctly_predicted_sense2count = k_Means.accuracy()
            if _ == n_repeat-1:
                ext.kmeans_display_at_instance_time(instance, data_size)
            n_repeat_micro_averages.append(sum(correctly_predicted_sense2count.values())/num_examples)
            n_repeat_macro_averages.append(sum(sense2f1.values())/len(sense2f1.values()))
        mean_macro_average, mean_micro_average = get_mean_metrics(n_repeat_macro_averages, n_repeat_micro_averages, display=True)
        n_repeat_most_frequent_senses = [ext.get_most_frequent_sense()[1], ext.get_most_frequent_sense()[2]]
        averages2score = make_csv_per_instance(instance, n_repeat, data_size, mean_macro_average, mean_micro_average, averages2score, n_repeat_most_frequent_senses)
    mean_macro_averages_for_kmeans.append(sum(averages2score["macro_average"])/len(averages2score["macro_average"]))
    mean_micro_averages_for_kmeans.append(sum(averages2score["micro_average"])/len(averages2score["micro_average"]))
create_kmeans_mean_accuracies_csv(mean_averages2score,mean_macro_averages_for_kmeans, mean_micro_averages_for_kmeans, data_sizes, n_repeat)

--> Instance : aboutir
Proportion de données annotées considérées sur le corpus: 75.0 %
Répartition des sens: Counter({'3': 41, '4': 7, '2': 1, '1': 1})
Le sens le plus fréquent pour 'aboutir' est le sens 3 avec une proportion de 82.0 %

Macro-average : 0.29
Micro-average : 0.73

--------------------------------------------------

--> Instance : investir
Proportion de données annotées considérées sur le corpus: 75.0 %
Répartition des sens: Counter({'4': 34, '3': 11, '1': 4, '2': 1})
Le sens le plus fréquent pour 'investir' est le sens 4 avec une proportion de 68.0 %

Macro-average : 0.26
Micro-average : 0.55

--------------------------------------------------

--> Instance : traduire
Proportion de données annotées considérées sur le corpus: 75.0 %
Répartition des sens: Counter({'1': 26, '5': 15, '4': 9})
Le sens le plus fréquent pour 'traduire' est le sens 1 avec une proportion de 52.0 %

Macro-average : 0.62
Micro-average : 0.67

--------------------------------------------------

-->

In [36]:
# merge csv classifier et csv kmeans pour avoir les deux dans le même csv en alterant les colonnes de chacun des csv
# (so not this)
def merge_csv(classifier_csv, kmeans_csv):
    '''
    - Permet de fusionner les deux csv obtenus pour les deux méthodes de classification
      dans un seul csv.

    INPUT
    classifier_csv (str): nom du csv obtenu pour la méthode de classification
    kmeans_csv (str): nom du csv obtenu pour la méthode de kmeans

    OUTPUT
    merged_csv (str): nom du csv fusionné
    '''
    classifier_df = pd.read_csv(classifier_csv)
    kmeans_df = pd.read_csv(kmeans_csv)
    merged_df = pd.merge(classifier_df, kmeans_df, on="data_sizes")
    merged_df.to_csv("merged_df.csv")
    return merged_df 